In [ ]:
import tomopy

In [ ]:
import pyfits
import numpy as np
import os

In [ ]:
%gui qt

In [ ]:
from IPython.html import widgets
from IPython.display import display

In [ ]:
from PyQt4 import QtCore, QtGui

def gui_fname(dir=None):
    """Select a file via a dialog and returns the file name.
    """
    if dir is None: dir ='./'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, "Select Folder ...",
                                                  dir,
                                                  QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

In [ ]:
# load the OB

In [ ]:
%gui qt
ob_folder = str(gui_fname())
%matplotlib inline

In [ ]:
import glob

In [ ]:
print ob_folder

In [ ]:
from pprint import pprint

In [ ]:
pprint(glob.glob(ob_folder + '/*'))
list_ob_files = glob.glob(ob_folder + '/*')

In [ ]:
number_of_ob = len(list_ob_files)
print(number_of_ob)

In [ ]:
ob_data = []
for _file in list_ob_files:
    ob_data.append(pyfits.open(_file)[0].data)
    type(pyfits.open(_file))


In [ ]:
type(ob_data[0])

In [ ]:
ob_data[1]

In [ ]:
ob_data[0].shape

In [ ]:
# load the DF

In [ ]:
%gui qt
df_folder = str(gui_fname(ob_folder))
%matplotlib inline

In [ ]:
list_df_files = glob.glob(df_folder + '/*')

In [ ]:
df_data = []
for _file in list_df_files:
    df_data.append(pyfits.open(_file)[0].data)

In [ ]:
# load the data

In [ ]:
%gui qt
data_folder = str(gui_fname(df_folder))
%matplotlib inline

In [ ]:
def getFilename(filename_template, angle):
        path_pattern = filename_template % (angle,)
        dir = os.path.dirname(path_pattern)
        basename = os.path.basename(path_pattern)
        base, ext = os.path.splitext(basename)
        # bad code
        path_pattern = os.path.join(dir, base.replace(".", "[_.]") + ext)
        
        paths = glob.glob(path_pattern)
        if len(paths)!=1:
            raise RuntimeError, "template %r no good: \npath_pattern=%r" % (
                filename_template, path_pattern)
    
        path = paths[0]
        return path

In [ ]:
angle=0; getFilename(data_folder+"/2012*_%.3f_*.fits", angle)

In [ ]:
step = 0.85

In [ ]:
angles=np.arange(0.0, 181.9+step, step);
list_data_files = map(lambda x: getFilename(data_folder+"/2012*_%.3f_*.fits", x), angles);
pprint(list_data_files)
               

In [ ]:
data = []
for _file in list_data_files:
    data.append(pyfits.open(_file)[0].data)

In [ ]:
proj = tomopy.normalize(data, ob_data, df_data)

In [ ]:
proj.shape

In [ ]:
theta = np.pi/180.*(np.arange(0.0, 181.9+step, step))
print(np.arange(0.0, 181.9+step, step))

In [ ]:
pprint(theta)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

In [ ]:
# proj = np.r_[proj[0:70], proj[81:225]].copy()
#theta = np.r_[theta[0:70], theta[81:225]].copy()

In [ ]:
rot_center = tomopy.find_center(proj, theta, emission=False, init=1024, tol=0.5)
print("Center of rotation: ", rot_center)

In [ ]:
print proj.shape
tomopy.write_center(proj.copy(), theta, dpath='~/imaging/2nd-try/tmp', emission=False)

In [ ]:
print proj.shape
rot_center = tomopy.find_center_vo(proj.copy())
print("Center of rotation: ", rot_center)

In [ ]:
proj = tomopy.normalize_bg(data)

In [ ]:
fig, ax = plt.subplots()
im = plt.imshow(data[0], cmap='gray') #, vmin=0.2, vmax=1.2
fig.colorbar(im)

In [ ]:
fig, ax = plt.subplots()
im = plt.imshow(proj[:, 1000, :], cmap='gray')
fig.colorbar(im)

In [ ]:
tomopy.write_tiff_stack(proj, fname='~/imaging/2nd-try/tmp', axis=1, overwrite=True)

In [ ]:
rec = tomopy.recon(proj[:, 300:201, :], theta=theta, center=1024.4, algorithm='gridrec', emission=False)

fig, ax = plt.subplots()
im = plt.imshow(rec[0], cmap='gray')
fig.colorbar(im)

In [ ]:
print proj.shape, rec.shape
tomopy.write_tiff_stack(rec, fname='~/imaging/2nd-try/recon/tmp', axis=0, overwrite=True)